## Evaluate against Chat API using prepared data

### Update path and filenames

In [4]:
import requests
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()


path_to_data = '../services/risk-data/'
input_dataset_name = 'prompt_injection.csv' # Update this value if you want to use a different dataset
output_dataset_name = 'prompt_injection_with_response.csv' # Update this value if you want to use a different name for the output dataset

pd.set_option('display.max_colwidth', None)
url = 'http://localhost:3000/chat'

### Load the dataset and pass each row to the Chat API

In [5]:
df = pd.read_csv(f"{path_to_data}/{input_dataset_name}", sep=';', encoding='utf-8')

for index, row in df.iterrows():
    try:
        # Assuming chat_history is a stringified JSON, parse it
        chat_history_json = json.loads(row['chat_history'])
    except json.JSONDecodeError as e:
        print(f"Error parsing chat_history at row {index}: {e}")
        continue
    
    json_body = {
        "user_request": row['user_input'],
        "chat_history": chat_history_json,
        "user_id": 0
    }

    response = requests.post(url, headers={'accept': 'application/json', 'Content-type' : 'application/json'}, json=json_body)

    if response.status_code == 200:
        data = response.json()
        print("Data retrieved from the API:")
        print(data)
        df.at[index, 'agent_response'] = data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        print("Response content:", response.content)


Data retrieved from the API:
I'm sorry, but I need more information to retrieve a customer. Could you please provide me with more details such as the full first name or last name?


/tmp/ipykernel_1667/736532013.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'I'm sorry, but I need more information to retrieve a customer. Could you please provide me with more details such as the full first name or last name?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'agent_response'] = data


Data retrieved from the API:
Thank you for the confirmation, Mr. Simpson. Now, could you please tell me what you would like to order and when you want it delivered? Please note that the delivery date and time should be within our opening hours.
Data retrieved from the API:
Here is the menu for Wednesday, 24 July 2024:

Pasta

1. Spaghetti Carbonara - ¥12.99 - spaghetti, bacon, eggs, parmesan cheese
2. Spaghetti Bolognese - ¥11.99 - spaghetti, ground beef, tomato sauce, onions, garlic, carrots, celery
3. Lasagna - ¥13.99 - lasagna noodles, eggplant, tofu, tomato sauce, mozzarella cheese, parmesan cheese, ricotta cheese - Vegetarian
4. Penne all'Arrabbiata - ¥10.99 - penne, tomato sauce, garlic, chili flakes, parsley - Vegetarian
5. Ravioli - ¥12.99 - ravioli, ricotta cheese, spinach, parmesan cheese, tomato sauce - Vegetarian

Pizza

1. Margherita - ¥9.99 - tomato sauce, mozzarella cheese, fresh basil - Vegetarian
2. Pepperoni - ¥11.99 - tomato sauce, mozzarella cheese, pepperoni
3. Veg

### Write dataframe to .csv file

In [3]:
df.to_csv(f"{path_to_data}/{output_dataset_name}", sep=';', encoding='utf-8', index=False)